In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import random
import uuid

In [2]:
from matplotlib import rc
rc('animation', html='jshtml')

In [3]:
INIT_PREDATORS = 100        #Początkowa liczba drapieżników
INIT_PREYS = 500            #Początkowa liczba ofiar
X_MIN = 0
X_MAX = 2
Y_MIN = 0
Y_MAX = 2
SAFE_DISTANCE = 0.1         #odległość potrzebna do ataku drapieżnika
PREDATOR_EFFECTIVENESS = 30 #liczba z zakresu od 0-100 określająca szansę na śmierć ofiary w starciu z drapieżnikiem
MAX_VITALITY = 30           #wytrzymałość drapieżników (ile rund jest w stanie wytrzymać bez jedzenia)
ITERATIONS = 500            #liczba iteracji / kroków w błądzeniu losowym, którą symulujemy
STEP_SIZE = 0.02

In [4]:
class SurfaceBoundaries:
    def __init__(self, x_min, x_max, y_min, y_max):
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        
    def isWithinBoundaries(self, position):
        return (self.x_min <= position.x <= self.x_max and self.y_min <= position.y <= self.y_max)

In [5]:
surfaceBoundaries = SurfaceBoundaries(X_MIN, X_MAX, Y_MIN, Y_MAX)

In [6]:
class Position:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __str__(self):
        return ("Position(" + str(self.x) + ", " + str(self.y) + ")")
    
    def adjust_to_boundaries(self, surfaceBoundaries):
        if not surfaceBoundaries.isWithinBoundaries(self):
            if self.x < surfaceBoundaries.x_min: self.x = surfaceBoundaries.x_min
            if self.x > surfaceBoundaries.x_max: self.x = surfaceBoundaries.x_max
            if self.y < surfaceBoundaries.y_min: self.y = surfaceBoundaries.y_min
            if self.y > surfaceBoundaries.y_max: self.y = surfaceBoundaries.y_max
                
    def random_step(self, surfaceBoundaries):
        phi = 2 * np.pi * random.uniform(0, 1)
        self.x += np.cos(phi) * STEP_SIZE
        self.y += np.sin(phi) * STEP_SIZE
        self.adjust_to_boundaries(surfaceBoundaries)

def randomPosition(surfaceBoundaries):
    x = random.uniform(surfaceBoundaries.x_min, surfaceBoundaries.x_max)
    y = random.uniform(surfaceBoundaries.y_min, surfaceBoundaries.y_max)
    return Position(x, y)

def distance(position_a, position_b):
    return math.sqrt(
        pow(position_a.x - position_b.x, 2) + pow(position_a.y - position_b.y, 2)        
    )

In [7]:
def are_too_close(predator, prey):
            distance(prey.position, predator.position) < SAFE_DISTANCE
        
def coin_flip(probability):
    #accepts probability as number from 0 to 1
    #perform the binomial distribution (returns 0 or 1)    
    return np.random.binomial(1,probability)

In [11]:
class Predator:
    def __init__(self, surfaceBoundaries, MAX_VITALITY):
        self.id = uuid.uuid1()
        self.position = randomPosition(surfaceBoundaries)
        self.isAlive = True
        self.vitality = MAX_VITALITY

    def __str__(self):
        return ("Predator(" + 
                str(self.id) + ", " + 
                str(self.position) + ", " + 
                str(self.isAlive) + ", " + 
                str(self.vitality) + 
                ")")

In [26]:
class Prey:
    def __init__(self, surfaceBoundaries):
        self.id = uuid.uuid1()
        self.position = randomPosition(surfaceBoundaries)
        self.isAlive = True
        
    def get_endangering_predators(self, predators):
        return (self, list(filter(lambda predator: are_too_close(self, predator), predators)))
        
    def chance_to_die(self, endangering_predators_number):
        chance_to_survive = pow(((100 - PREDATOR_EFFECTIVENESS) / 100), endangering_predators_number)
        return 1 - chance_to_survive
    
    def __str__(self):
        return ("Prey(" + 
                str(self.id) + ", " + 
                str(self.position) + ", " + 
                str(self.isAlive) +
                ")")

In [27]:
predators = []
for i in range(INIT_PREDATORS):
    predators.append(Predator(surfaceBoundaries, MAX_VITALITY))

In [28]:
preys = []
for i in range(INIT_PREYS):
    preys.append(Prey(surfaceBoundaries))

In [29]:
dead_predators = []
dead_preys = []

In [30]:
def find_endangered_preys_and_attacking_predators():
    endangered_preys = []
    for prey in preys:
        endangering_predators = prey.get_endangering_predators(predators)
        
        if len(endangering_predators) > 0:
            chance_to_die = prey.chance_to_die(len(endangering_predators))
            endangered_preys.append((prey, endangering_predators, chance_to_die))
    return endangered_preys

In [31]:
def mark_predators_meal(happy_predators):
    updated_predators = []
    for predator in predators:
        if predator in happy_predators:
            predator.vitality = MAX_VITALITY
        updated_predators.append(predator)
    predators = updated_predators

In [32]:
def clash_preys_and_predators(endangered_preys):
    for (prey, endangering_predators, chance_to_die) in endangered_preys:
        prey_dies = (coin_flip(chance_to_die) == 1) # == 1 converts 0/1 to False/True
        if prey_dies:
            println("Prey " + str(prey) + " got killed by predators " + str(endangering_predators))
            prey.isAlive = False
            mark_predators_meal(endangering_predators)


In [33]:
def decrement_predators_vitality():
    for predator in predators:
        predator.vitality =+ -1
        if predator.vitality == 0:
            predator.isAlive = False

In [34]:
def filter_out_dead_predators():
    new_dead_predators = list(filter(lambda predator: (predator.isAlive == False), predators))
    alive_predators = list(filter(lambda predator: (predator.isAlive == True), predators))
    predators = alive_predators
    dead_predators += new_dead_predators    

In [35]:
def filter_out_dead_preys():
    new_killed_preys = list(filter(lambda prey: (prey.isAlive == False), preys))
    alive_preys = list(filter(lambda prey: (prey.isAlive == True), preys))
    preys = alive_preys
    dead_preys += new_killed_preys

In [36]:
def move_alive_animals_by_one_random_step():
    for predator in predators:
        predator.position.random_step(surfaceBoundaries)
    for prey in preys:
        prey.position.random_step(surfaceBoundaries)    

In [37]:
for i in range(ITERATIONS):
    endangered_preys = find_endangered_preys_and_attacking_predators()
    clash_preys_and_predators(endangered_preys)
    decrement_predators_vitality()
    filter_out_dead_predators()
    filter_out_dead_preys()
    move_alive_animals_by_one_random_step()
    

NameError: name 'math' is not defined